# Example Usage of Spike Detector

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
from ieeg.auth import Session

In [2]:
# Import custom functions
from get_iEEG_data import *
from spike_detector import *
from iEEG_helper_functions import *

## Select a Patient

In [3]:
hup_id = 166
dataset_name = "HUP166_phaseII"

## Create iEEG Session
**Note:** Use your own iEEG.org login .bin file!

In [4]:
with open("agu_ieeglogin.bin", "r") as f:
    session = Session("aguilac", f.read())

## Open Dataset

In [5]:
dataset = session.open_dataset(dataset_name)
all_channel_labels = np.array(dataset.get_channel_labels())

## Channel Selection

In [6]:
channel_labels_to_download = all_channel_labels[electrode_selection(all_channel_labels)]
channel_labels_to_download

array(['LA1', 'LA10', 'LA11', 'LA2', 'LA3', 'LA4', 'LA5', 'LA6', 'LA7',
       'LA8', 'LA9', 'LB0', 'LB1', 'LB11', 'LB12', 'LB2', 'LB3', 'LB4',
       'LB5', 'LB6', 'LB7', 'LB8', 'LB9', 'LC1', 'LC10', 'LC11', 'LC2',
       'LC3', 'LC4', 'LC5', 'LC6', 'LC7', 'LC8', 'LC9', 'LD1', 'LD2',
       'LD3', 'LD4', 'LD5', 'LD6', 'LD7', 'LD8', 'LE1', 'LE10', 'LE11',
       'LE12', 'LE2', 'LE3', 'LE4', 'LE5', 'LE6', 'LE7', 'LE8', 'LE9',
       'LF1', 'LF2', 'LF3', 'LF4', 'LF5', 'LF6', 'LF7', 'LF8', 'LG1',
       'LG10', 'LG11', 'LG12', 'LG2', 'LG3', 'LG4', 'LG5', 'LG6', 'LG7',
       'LG8', 'LG9', 'LH1', 'LH10', 'LH11', 'LH12', 'LH2', 'LH3', 'LH4',
       'LH5', 'LH6', 'LH7', 'LH8', 'LH9', 'LI1', 'LI2', 'LI3', 'LI4',
       'LI5', 'LI6', 'LI7', 'LI8', 'LJ1', 'LJ10', 'LJ11', 'LJ12', 'LJ2',
       'LJ3', 'LJ4', 'LJ5', 'LJ6', 'LJ7', 'LJ8', 'LJ9', 'LK1', 'LK10',
       'LK11', 'LK12', 'LK2', 'LK3', 'LK4', 'LK5', 'LK6', 'LK7', 'LK8',
       'LK9', 'LL1', 'LL2', 'LL3', 'LL4', 'LL5', 'LL6', 'LL7', 'LL8',

## Select a 1-Minute Window

In [7]:
duration_usec = 6e7  # 1 minute
start_time_usec = 1000 * 6e7  # 1 minutes in microseconds
stop_time_usec = start_time_usec + duration_usec
start_time_usec, stop_time_usec

(60000000000.0, 60060000000.0)

## Download iEEG Data

In [8]:
ieeg_data, fs = get_iEEG_data(
    "aguilac",
    "agu_ieeglogin.bin",
    dataset_name,
    start_time_usec,
    stop_time_usec,
    channel_labels_to_download,
)
fs = int(fs)
ieeg_data

,LA1,LA10,LA11,LA2,LA3,LA4,LA5,LA6,LA7,LA8,...,RC1,RC10,RC2,RC3,RC4,RC5,RC6,RC7,RC8,RC9
0,-13.823612,-316.879714,-163.224953,132.387666,99.689507,44.395061,-1.860871,-9.570193,-24.457159,9.304354,...,22.064611,260.787751,15.684482,67.257188,18.874547,-9.836031,3.721742,7.709322,9.304354,-0.265839
1,-10.101870,-314.487166,-164.554147,136.109407,102.082056,47.319286,1.063355,-9.038515,-20.203740,13.557773,...,27.115546,76.827380,21.001256,75.232348,24.457159,-0.797516,7.975161,11.696902,12.494418,-9.038515
2,-4.253419,-309.170392,-157.376502,141.957859,107.664668,51.041028,3.721742,-5.050935,-16.481999,18.874547,...,28.976417,-52.901899,23.393804,79.751606,27.381385,4.519258,11.696902,15.684482,17.013676,-6.380128
3,0.797516,-300.663554,-146.211278,145.679600,108.728023,53.965253,6.380128,-0.265839,-13.291934,22.862127,...,34.824868,-182.897016,27.647223,84.802541,27.381385,11.696902,13.823612,17.545353,18.077031,-10.101870
4,6.911806,-301.195232,-152.325567,150.198858,112.715603,57.421156,10.633547,-0.265839,-10.899386,23.393804,...,42.534190,-359.945581,34.824868,91.182669,28.710578,20.203740,15.950321,20.203740,20.469579,-16.747837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61435,-72.308123,180.504468,67.788865,-80.017445,-106.335475,-68.320542,-95.701927,-109.259700,-109.525539,-97.562798,...,-145.945439,-414.442512,39.078287,-81.346638,-37.749093,-212.670949,-59.282027,-19.406224,-22.064611,-36.154061
61436,-76.561542,174.921856,58.484511,-81.346638,-108.993861,-69.649736,-99.157830,-113.778958,-113.778958,-102.879572,...,-147.274632,-325.652391,33.495674,-81.346638,-38.812448,-210.810078,-60.611221,-19.937901,-22.862127,-30.837288
61437,-81.080799,169.073405,50.243512,-86.397573,-114.576474,-77.093219,-106.069636,-118.564054,-120.690764,-109.525539,...,-148.603826,-168.275889,24.988837,-82.675831,-37.217416,-210.810078,-61.142898,-21.267095,-23.659643,-17.013676
61438,-84.536702,166.680856,44.129222,-89.055960,-118.564054,-79.219929,-107.664668,-120.956602,-122.817473,-112.981442,...,-153.388922,-9.570193,16.481999,-85.600057,-36.685739,-214.265981,-62.206253,-22.330450,-22.862127,-14.355289


## Artifact Rejection

In [9]:
good_channels_res = detect_bad_channels_optimized(ieeg_data.to_numpy(), fs)
good_channel_indicies = good_channels_res[0]
good_channel_labels = channel_labels_to_download[good_channel_indicies]
ieeg_data = ieeg_data[good_channel_labels].to_numpy()

## Common Average Reference

In [10]:
ieeg_data = common_average_montage(ieeg_data)

## Line Noise Removal

In [11]:
ieeg_data = notch_filter(ieeg_data, 59, 61, fs)
ieeg_data = bandpass_filter(ieeg_data, 1, 70, fs)

## Spike Detection

In [12]:
spike_output = spike_detector(
    data=ieeg_data,
    fs=fs,
    electrode_labels=good_channel_labels,
)
spike_output = spike_output.astype(int)
spike_output

array([[ 6084,    42,     0],
       [ 6114,    47,     0],
       [ 6116,    48,     0],
       [ 6116,    46,     0],
       [18731,    10,     1],
       [18759,     2,     1],
       [20252,     0,     2],
       [20254,     3,     2],
       [58945,     1,     3],
       [58949,    10,     3]])

In [13]:
# Put spike output into a dataframe with the first column being peak_index, second column being channel_label, third column being sequence_index
spike_output_df = pd.DataFrame(
    spike_output, columns=["peak_index", "channel_label", "sequence_index"]
)
spike_output_df

,peak_index,channel_label,sequence_index
0,6084,42,0
1,6114,47,0
2,6116,48,0
3,6116,46,0
4,18731,10,1
5,18759,2,1
6,20252,0,2
7,20254,3,2
8,58945,1,3
9,58949,10,3
